In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt

In [2]:
with open("vocab.distances.json", "r") as f:
    vocab_distances = json.load(f)

In [3]:
id_name_mapping = {}

heuristic = []
for row in vocab_distances:
    
    id_name_mapping[row["word_id"]] = row["word_name"]
    id_name_mapping[row["word_name"]] = row["word_id"]
    
    current_word = {
        "word_name": row["word_name"],
        "heuristic": 0.0}
    
    for n in row["neighbors"][1:]:
        current_word["heuristic"] += n["word_distance"]
    heuristic.append(current_word)
    row["heuristic"] = current_word["heuristic"]

In [4]:
for row in vocab_distances[20:25]:
    print("Word: \"%s\" Heuristic: %.3f" % (row["word_name"], row["heuristic"]))
    for n in row["neighbors"][1:]:
        print("    Neighbor: \"%s\" Distance: %.3f" % (n["word_name"], n["word_distance"]))

Word: "are" Heuristic: 72.327
    Neighbor: "is" Distance: 3.122
    Neighbor: "was" Distance: 3.394
    Neighbor: "were" Distance: 3.501
    Neighbor: "be" Distance: 3.733
    Neighbor: "thats" Distance: 3.796
    Neighbor: "completely" Distance: 3.831
    Neighbor: "'re" Distance: 3.869
    Neighbor: "clearly" Distance: 3.875
    Neighbor: "neatly" Distance: 3.888
    Neighbor: "slightly" Distance: 3.896
    Neighbor: "nearly" Distance: 3.909
    Neighbor: "almost" Distance: 3.916
    Neighbor: "currently" Distance: 3.926
    Neighbor: "ladies" Distance: 3.932
    Neighbor: "others" Distance: 3.934
    Neighbor: "nicely" Distance: 3.941
    Neighbor: "happily" Distance: 3.949
    Neighbor: "seemingly" Distance: 3.956
    Neighbor: "jetliner" Distance: 3.958
Word: "next" Heuristic: 77.120
    Neighbor: "adjacent" Distance: 3.656
    Neighbor: "attached" Distance: 3.857
    Neighbor: "connected" Distance: 3.915
    Neighbor: "affixed" Distance: 3.965
    Neighbor: "tending" Distance: 3

In [5]:
heuristic = sorted(heuristic, key=(lambda x: -x["heuristic"]))
for row in heuristic:
    print("Word: \"%s\" Heuristic: %.3f" % (row["word_name"], row["heuristic"]))

Word: "." Heuristic: 137.193
Word: "very" Heuristic: 105.334
Word: "that" Heuristic: 102.010
Word: "an" Heuristic: 100.538
Word: "," Heuristic: 98.971
Word: "some" Heuristic: 91.475
Word: "one" Heuristic: 91.262
Word: "while" Heuristic: 89.728
Word: "there" Heuristic: 89.391
Word: "and" Heuristic: 88.491
Word: "this" Heuristic: 86.734
Word: "``" Heuristic: 86.152
Word: "each" Heuristic: 85.747
Word: "as" Heuristic: 85.476
Word: "wearing" Heuristic: 84.990
Word: "during" Heuristic: 81.825
Word: "being" Heuristic: 81.596
Word: "at" Heuristic: 81.199
Word: "'s" Heuristic: 80.874
Word: "in" Heuristic: 78.502
Word: "all" Heuristic: 78.470
Word: "looking" Heuristic: 77.408
Word: "from" Heuristic: 77.323
Word: "next" Heuristic: 77.120
Word: "playing" Heuristic: 76.343
Word: "hot" Heuristic: 75.937
Word: "of" Heuristic: 75.855
Word: "close" Heuristic: 75.648
Word: "covered" Heuristic: 74.870
Word: "has" Heuristic: 73.868
Word: "for" Heuristic: 73.478
Word: "is" Heuristic: 73.012
Word: "are" He

Word: "pitch" Heuristic: 29.929
Word: "neath" Heuristic: 29.926
Word: "lines" Heuristic: 29.924
Word: "frosting" Heuristic: 29.923
Word: "arrangement" Heuristic: 29.919
Word: "whip" Heuristic: 29.919
Word: "patterned" Heuristic: 29.913
Word: "let" Heuristic: 29.912
Word: "candle" Heuristic: 29.912
Word: "extending" Heuristic: 29.910
Word: "studio" Heuristic: 29.906
Word: "windows" Heuristic: 29.901
Word: "surround" Heuristic: 29.898
Word: "balanced" Heuristic: 29.893
Word: "glowing" Heuristic: 29.883
Word: "least" Heuristic: 29.883
Word: "freight" Heuristic: 29.881
Word: "bread" Heuristic: 29.880
Word: "poles" Heuristic: 29.876
Word: "ducks" Heuristic: 29.875
Word: "herself" Heuristic: 29.873
Word: "protective" Heuristic: 29.865
Word: "pitchers" Heuristic: 29.865
Word: "delivering" Heuristic: 29.863
Word: "pancakes" Heuristic: 29.863
Word: "plowing" Heuristic: 29.862
Word: "volleyball" Heuristic: 29.853
Word: "did" Heuristic: 29.852
Word: "figure" Heuristic: 29.851
Word: "spewing" Heur

Word: "kreme" Heuristic: 27.367
Word: "supposed" Heuristic: 27.367
Word: "licks" Heuristic: 27.366
Word: "encased" Heuristic: 27.366
Word: "external" Heuristic: 27.365
Word: "circle" Heuristic: 27.365
Word: "non" Heuristic: 27.364
Word: "weaving" Heuristic: 27.364
Word: "entrees" Heuristic: 27.363
Word: "cathedral" Heuristic: 27.362
Word: "pad" Heuristic: 27.362
Word: "arrives" Heuristic: 27.359
Word: "mustache" Heuristic: 27.359
Word: "sucking" Heuristic: 27.359
Word: "retaining" Heuristic: 27.359
Word: "favorite" Heuristic: 27.358
Word: "stony" Heuristic: 27.358
Word: "cucumber" Heuristic: 27.357
Word: "futon" Heuristic: 27.357
Word: "treats" Heuristic: 27.356
Word: "performer" Heuristic: 27.356
Word: "transports" Heuristic: 27.355
Word: "forehand" Heuristic: 27.355
Word: "keeps" Heuristic: 27.355
Word: "stew" Heuristic: 27.354
Word: "hook" Heuristic: 27.354
Word: "helmeted" Heuristic: 27.353
Word: "costumed" Heuristic: 27.353
Word: "disposable" Heuristic: 27.352
Word: "cgi" Heuristi

Word: "marriage" Heuristic: 26.531
Word: "offspring" Heuristic: 26.531
Word: "peacefully" Heuristic: 26.531
Word: "wrench" Heuristic: 26.531
Word: "turbine" Heuristic: 26.531
Word: "grated" Heuristic: 26.531
Word: "plentiful" Heuristic: 26.531
Word: "girafee" Heuristic: 26.530
Word: "anticipation" Heuristic: 26.530
Word: "beet" Heuristic: 26.530
Word: "ascend" Heuristic: 26.529
Word: "mandarin" Heuristic: 26.529
Word: "rod" Heuristic: 26.529
Word: "oceans" Heuristic: 26.529
Word: "anticipates" Heuristic: 26.529
Word: "landscaped" Heuristic: 26.529
Word: "barefooted" Heuristic: 26.529
Word: "express" Heuristic: 26.529
Word: "posting" Heuristic: 26.529
Word: "reins" Heuristic: 26.528
Word: "sponsored" Heuristic: 26.528
Word: "man-made" Heuristic: 26.528
Word: "vivid" Heuristic: 26.528
Word: "republican" Heuristic: 26.528
Word: "correct" Heuristic: 26.528
Word: "network" Heuristic: 26.527
Word: "gearing" Heuristic: 26.527
Word: "snowshoes" Heuristic: 26.527
Word: "knive" Heuristic: 26.527

Word: "unhealthy" Heuristic: 26.155
Word: "<UNK>" Heuristic: 26.155
Word: "scarfs" Heuristic: 26.155
Word: "expressions" Heuristic: 26.155
Word: "beachside" Heuristic: 26.155
Word: "colleagues" Heuristic: 26.155
Word: "junked" Heuristic: 26.155
Word: "headgear" Heuristic: 26.154
Word: "dials" Heuristic: 26.154
Word: "af" Heuristic: 26.154
Word: "hammock" Heuristic: 26.154
Word: "vows" Heuristic: 26.154
Word: "thai" Heuristic: 26.154
Word: "compare" Heuristic: 26.154
Word: "accepts" Heuristic: 26.154
Word: "feather" Heuristic: 26.154
Word: "<UNK>" Heuristic: 26.153
Word: "steams" Heuristic: 26.153
Word: "tom" Heuristic: 26.153
Word: "walsk" Heuristic: 26.153
Word: "dangled" Heuristic: 26.153
Word: "panties" Heuristic: 26.153
Word: "2009" Heuristic: 26.153
Word: "<UNK>" Heuristic: 26.152
Word: "roaster" Heuristic: 26.152
Word: "stret" Heuristic: 26.152
Word: "clasped" Heuristic: 26.152
Word: "<UNK>" Heuristic: 26.152
Word: "creatures" Heuristic: 26.152
Word: "courses" Heuristic: 26.152
W

Word: "microscope" Heuristic: 25.748
Word: "<UNK>" Heuristic: 25.748
Word: "brands" Heuristic: 25.748
Word: "<UNK>" Heuristic: 25.747
Word: "poop" Heuristic: 25.747
Word: "tigers" Heuristic: 25.747
Word: "steadily" Heuristic: 25.747
Word: "grapefruits" Heuristic: 25.747
Word: "graduates" Heuristic: 25.747
Word: "fudge" Heuristic: 25.746
Word: "<UNK>" Heuristic: 25.746
Word: "<UNK>" Heuristic: 25.746
Word: "zones" Heuristic: 25.745
Word: "offset" Heuristic: 25.744
Word: "goalkeeper" Heuristic: 25.744
Word: "fiction" Heuristic: 25.744
Word: "<UNK>" Heuristic: 25.744
Word: "tub/shower" Heuristic: 25.744
Word: "smallest" Heuristic: 25.743
Word: "<UNK>" Heuristic: 25.743
Word: "decline" Heuristic: 25.743
Word: "chamber" Heuristic: 25.743
Word: "joker" Heuristic: 25.742
Word: "wristband" Heuristic: 25.742
Word: "once" Heuristic: 25.742
Word: "<UNK>" Heuristic: 25.741
Word: "forty-five" Heuristic: 25.741
Word: "candlesticks" Heuristic: 25.741
Word: "forwards" Heuristic: 25.741
Word: "pda" Heu

In [6]:
heuristic_by_id = sorted(heuristic, key=(lambda x: id_name_mapping[x["word_name"]]))
for row in heuristic_by_id:
    print("ID: %d Word: \"%s\" Heuristic: %.3f" % (id_name_mapping[row["word_name"]], row["word_name"], row["heuristic"]))

ID: 0 Word: "a" Heuristic: 55.777
ID: 1 Word: "<S>" Heuristic: 19.683
ID: 2 Word: "</S>" Heuristic: 26.196
ID: 3 Word: "." Heuristic: 137.193
ID: 4 Word: "on" Heuristic: 71.413
ID: 5 Word: "of" Heuristic: 75.855
ID: 6 Word: "the" Heuristic: 70.459
ID: 7 Word: "in" Heuristic: 78.502
ID: 8 Word: "with" Heuristic: 63.481
ID: 9 Word: "and" Heuristic: 88.491
ID: 10 Word: "is" Heuristic: 73.012
ID: 11 Word: "man" Heuristic: 51.308
ID: 12 Word: "to" Heuristic: 57.688
ID: 13 Word: "sitting" Heuristic: 59.773
ID: 14 Word: "an" Heuristic: 100.538
ID: 15 Word: "two" Heuristic: 62.589
ID: 16 Word: "," Heuristic: 98.971
ID: 17 Word: "standing" Heuristic: 59.375
ID: 18 Word: "at" Heuristic: 81.199
ID: 19 Word: "people" Heuristic: 51.694
ID: 20 Word: "are" Heuristic: 72.327
ID: 21 Word: "next" Heuristic: 77.120
ID: 22 Word: "white" Heuristic: 50.531
ID: 23 Word: "woman" Heuristic: 50.004
ID: 24 Word: "street" Heuristic: 48.236
ID: 25 Word: "table" Heuristic: 42.643
ID: 26 Word: "that" Heuristic: 102.

ID: 1354 Word: "cleaning" Heuristic: 33.457
ID: 1355 Word: "bare" Heuristic: 32.431
ID: 1356 Word: "blowing" Heuristic: 36.455
ID: 1357 Word: "goat" Heuristic: 28.579
ID: 1358 Word: "first" Heuristic: 33.891
ID: 1359 Word: "controls" Heuristic: 26.928
ID: 1360 Word: "laughing" Heuristic: 32.607
ID: 1361 Word: "advertisement" Heuristic: 31.039
ID: 1362 Word: "checking" Heuristic: 41.470
ID: 1363 Word: "straw" Heuristic: 31.400
ID: 1364 Word: "mobile" Heuristic: 33.778
ID: 1365 Word: "lighting" Heuristic: 30.761
ID: 1366 Word: "basketball" Heuristic: 34.362
ID: 1367 Word: "olives" Heuristic: 27.229
ID: 1368 Word: "displays" Heuristic: 34.779
ID: 1369 Word: "shape" Heuristic: 36.081
ID: 1370 Word: "cookies" Heuristic: 27.643
ID: 1371 Word: "great" Heuristic: 32.654
ID: 1372 Word: "onion" Heuristic: 33.769
ID: 1373 Word: "gravel" Heuristic: 33.960
ID: 1374 Word: "rusty" Heuristic: 30.490
ID: 1375 Word: "cardboard" Heuristic: 34.179
ID: 1376 Word: "fountain" Heuristic: 27.373
ID: 1377 Word:

ID: 2354 Word: "roller" Heuristic: 31.223
ID: 2355 Word: "teaching" Heuristic: 33.273
ID: 2356 Word: "lime" Heuristic: 33.977
ID: 2357 Word: "lining" Heuristic: 31.883
ID: 2358 Word: "cubs" Heuristic: 27.394
ID: 2359 Word: "possibly" Heuristic: 31.006
ID: 2360 Word: "altered" Heuristic: 31.977
ID: 2361 Word: "hiking" Heuristic: 31.965
ID: 2362 Word: "caption" Heuristic: 32.910
ID: 2363 Word: "breaking" Heuristic: 28.581
ID: 2364 Word: "pads" Heuristic: 27.837
ID: 2365 Word: "old-fashioned" Heuristic: 27.098
ID: 2366 Word: "navy" Heuristic: 29.193
ID: 2367 Word: "alarm" Heuristic: 32.848
ID: 2368 Word: "coast" Heuristic: 29.430
ID: 2369 Word: "forested" Heuristic: 29.731
ID: 2370 Word: "swung" Heuristic: 32.140
ID: 2371 Word: "youth" Heuristic: 31.720
ID: 2372 Word: "dancing" Heuristic: 28.339
ID: 2373 Word: "barren" Heuristic: 29.074
ID: 2374 Word: "harness" Heuristic: 27.009
ID: 2375 Word: "pork" Heuristic: 28.127
ID: 2376 Word: "falls" Heuristic: 30.634
ID: 2377 Word: "care" Heuristi

ID: 3853 Word: "paneling" Heuristic: 26.483
ID: 3854 Word: "parks" Heuristic: 27.280
ID: 3855 Word: "deliver" Heuristic: 28.286
ID: 3856 Word: "casting" Heuristic: 31.332
ID: 3857 Word: "dangling" Heuristic: 27.932
ID: 3858 Word: "boston" Heuristic: 27.107
ID: 3859 Word: "divider" Heuristic: 25.864
ID: 3860 Word: "receive" Heuristic: 27.269
ID: 3861 Word: "animated" Heuristic: 27.288
ID: 3862 Word: "laughs" Heuristic: 27.872
ID: 3863 Word: "boating" Heuristic: 28.204
ID: 3864 Word: "rounded" Heuristic: 27.548
ID: 3865 Word: "beam" Heuristic: 26.495
ID: 3866 Word: "boulders" Heuristic: 26.134
ID: 3867 Word: "highchair" Heuristic: 25.993
ID: 3868 Word: "maneuver" Heuristic: 26.557
ID: 3869 Word: "frisby" Heuristic: 27.313
ID: 3870 Word: "brownies" Heuristic: 26.633
ID: 3871 Word: "tone" Heuristic: 27.060
ID: 3872 Word: "peer" Heuristic: 28.670
ID: 3873 Word: "squat" Heuristic: 29.223
ID: 3874 Word: "foamy" Heuristic: 27.664
ID: 3875 Word: "hammock" Heuristic: 26.154
ID: 3876 Word: "trian

ID: 4353 Word: "gummy" Heuristic: 31.942
ID: 4354 Word: "celebrate" Heuristic: 26.934
ID: 4355 Word: "overview" Heuristic: 27.745
ID: 4356 Word: "ostriches" Heuristic: 26.191
ID: 4357 Word: "maple" Heuristic: 27.420
ID: 4358 Word: "amenities" Heuristic: 26.735
ID: 4359 Word: "moss" Heuristic: 28.501
ID: 4360 Word: "'re" Heuristic: 28.646
ID: 4361 Word: "exercise" Heuristic: 28.222
ID: 4362 Word: "francisco" Heuristic: 26.615
ID: 4363 Word: "billowing" Heuristic: 27.488
ID: 4364 Word: "crashed" Heuristic: 28.075
ID: 4365 Word: "hilltop" Heuristic: 26.139
ID: 4366 Word: "hairdryer" Heuristic: 26.219
ID: 4367 Word: "cheek" Heuristic: 26.209
ID: 4368 Word: "repairing" Heuristic: 27.504
ID: 4369 Word: "auditorium" Heuristic: 26.853
ID: 4370 Word: "donkeys" Heuristic: 26.906
ID: 4371 Word: "temple" Heuristic: 26.665
ID: 4372 Word: "await" Heuristic: 28.033
ID: 4373 Word: "testing" Heuristic: 28.124
ID: 4374 Word: "less" Heuristic: 27.399
ID: 4375 Word: "brownie" Heuristic: 27.078
ID: 4376 Wo

...

ID: 6852 Word: "nypd" Heuristic: 26.095
ID: 6853 Word: "varied" Heuristic: 26.852
ID: 6854 Word: "y" Heuristic: 26.887
ID: 6855 Word: "locomotives" Heuristic: 26.352
ID: 6856 Word: "tranquil" Heuristic: 26.889
ID: 6857 Word: "elizabeth" Heuristic: 25.878
ID: 6858 Word: "detroit" Heuristic: 26.487
ID: 6859 Word: "pinstripe" Heuristic: 26.371
ID: 6860 Word: "beaded" Heuristic: 26.376
ID: 6861 Word: "traveled" Heuristic: 26.569
ID: 6862 Word: "daughters" Heuristic: 26.583
ID: 6863 Word: "nick" Heuristic: 26.488
ID: 6864 Word: "tack" Heuristic: 25.675
ID: 6865 Word: "coordinating" Heuristic: 26.750
ID: 6866 Word: "joy" Heuristic: 25.584
ID: 6867 Word: "largest" Heuristic: 25.965
ID: 6868 Word: "stir-fry" Heuristic: 26.247
ID: 6869 Word: "writings" Heuristic: 26.296
ID: 6870 Word: "tether" Heuristic: 25.586
ID: 6871 Word: "sippy" Heuristic: 27.875
ID: 6872 Word: "tigers" Heuristic: 25.747
ID: 6873 Word: "breath" Heuristic: 26.236
ID: 6874 Word: "rooftops" Heuristic: 26.371
ID: 6875 Word: "r

ID: 8851 Word: "chilies" Heuristic: 25.769
ID: 8852 Word: "rump" Heuristic: 26.627
ID: 8853 Word: "cathay" Heuristic: 26.308
ID: 8854 Word: "happens" Heuristic: 26.604
ID: 8855 Word: "screwed" Heuristic: 26.877
ID: 8856 Word: "nathans" Heuristic: 26.293
ID: 8857 Word: "feathery" Heuristic: 26.075
ID: 8858 Word: "limousine" Heuristic: 26.474
ID: 8859 Word: "saber" Heuristic: 25.677
ID: 8860 Word: "overheard" Heuristic: 26.396
ID: 8861 Word: "attractions" Heuristic: 25.794
ID: 8862 Word: "fantasy" Heuristic: 25.959
ID: 8863 Word: "handkerchief" Heuristic: 25.949
ID: 8864 Word: "thorough" Heuristic: 26.067
ID: 8865 Word: "minnesota" Heuristic: 26.739
ID: 8866 Word: "despite" Heuristic: 26.273
ID: 8867 Word: "toolbox" Heuristic: 26.112
ID: 8868 Word: "solders" Heuristic: 26.127
ID: 8869 Word: "shortly" Heuristic: 26.455
ID: 8870 Word: "bathmat" Heuristic: 25.463
ID: 8871 Word: "duke" Heuristic: 26.689
ID: 8872 Word: "loitering" Heuristic: 25.823
ID: 8873 Word: "whle" Heuristic: 26.147
ID: 

ID: 9351 Word: "arc" Heuristic: 26.179
ID: 9352 Word: "glistening" Heuristic: 26.404
ID: 9353 Word: "rip" Heuristic: 26.077
ID: 9354 Word: "currents" Heuristic: 25.370
ID: 9355 Word: "nosing" Heuristic: 25.932
ID: 9356 Word: "half-filled" Heuristic: 26.476
ID: 9357 Word: "shoved" Heuristic: 26.731
ID: 9358 Word: "brochure" Heuristic: 26.422
ID: 9359 Word: "bye" Heuristic: 26.168
ID: 9360 Word: "myspace" Heuristic: 25.794
ID: 9361 Word: "pealed" Heuristic: 25.997
ID: 9362 Word: "divers" Heuristic: 26.329
ID: 9363 Word: "tolit" Heuristic: 26.140
ID: 9364 Word: "fied" Heuristic: 26.106
ID: 9365 Word: "readied" Heuristic: 27.173
ID: 9366 Word: "leak" Heuristic: 25.918
ID: 9367 Word: "murdered" Heuristic: 25.871
ID: 9368 Word: "broiler" Heuristic: 26.239
ID: 9369 Word: "broiled" Heuristic: 25.965
ID: 9370 Word: "climbed" Heuristic: 26.225
ID: 9371 Word: "schoolboys" Heuristic: 26.359
ID: 9372 Word: "copse" Heuristic: 26.840
ID: 9373 Word: "sixteen" Heuristic: 27.009
ID: 9374 Word: "competes

ID: 11350 Word: "scrambles" Heuristic: 25.932
ID: 11351 Word: "creamed" Heuristic: 26.300
ID: 11352 Word: "pirched" Heuristic: 25.806
ID: 11353 Word: "two-person" Heuristic: 26.164
ID: 11354 Word: "eclairs" Heuristic: 25.827
ID: 11355 Word: "twitter" Heuristic: 25.984
ID: 11356 Word: "humorously" Heuristic: 25.913
ID: 11357 Word: "usaf" Heuristic: 26.197
ID: 11358 Word: "offset" Heuristic: 25.744
ID: 11359 Word: "strategy" Heuristic: 25.735
ID: 11360 Word: "poinsettia" Heuristic: 26.008
ID: 11361 Word: "hosting" Heuristic: 26.740
ID: 11362 Word: "intersects" Heuristic: 26.266
ID: 11363 Word: "penne" Heuristic: 26.043
ID: 11364 Word: "obstruction" Heuristic: 25.868
ID: 11365 Word: "trousers" Heuristic: 25.859
ID: 11366 Word: "coin-operated" Heuristic: 26.238
ID: 11367 Word: "permit" Heuristic: 26.037
ID: 11368 Word: "pecan" Heuristic: 26.446
ID: 11369 Word: "gasoline" Heuristic: 25.873
ID: 11370 Word: "scoring" Heuristic: 25.821
ID: 11371 Word: "fiery" Heuristic: 25.886
ID: 11372 Word: 

ID: 11999 Word: "<UNK>" Heuristic: 25.795
ID: 11999 Word: "<UNK>" Heuristic: 25.794
ID: 11999 Word: "<UNK>" Heuristic: 25.788
ID: 11999 Word: "<UNK>" Heuristic: 25.787
ID: 11999 Word: "<UNK>" Heuristic: 25.786
ID: 11999 Word: "<UNK>" Heuristic: 25.786
ID: 11999 Word: "<UNK>" Heuristic: 25.786
ID: 11999 Word: "<UNK>" Heuristic: 25.784
ID: 11999 Word: "<UNK>" Heuristic: 25.784
ID: 11999 Word: "<UNK>" Heuristic: 25.783
ID: 11999 Word: "<UNK>" Heuristic: 25.781
ID: 11999 Word: "<UNK>" Heuristic: 25.776
ID: 11999 Word: "<UNK>" Heuristic: 25.772
ID: 11999 Word: "<UNK>" Heuristic: 25.770
ID: 11999 Word: "<UNK>" Heuristic: 25.766
ID: 11999 Word: "<UNK>" Heuristic: 25.765
ID: 11999 Word: "<UNK>" Heuristic: 25.761
ID: 11999 Word: "<UNK>" Heuristic: 25.761
ID: 11999 Word: "<UNK>" Heuristic: 25.760
ID: 11999 Word: "<UNK>" Heuristic: 25.760
ID: 11999 Word: "<UNK>" Heuristic: 25.758
ID: 11999 Word: "<UNK>" Heuristic: 25.748
ID: 11999 Word: "<UNK>" Heuristic: 25.747
ID: 11999 Word: "<UNK>" Heuristic:

In [13]:
heuristic_array = np.array([x["heuristic"] for x in heuristic_by_id])
heuristic_array = heuristic_array - heuristic_array.min()
heuristic_array = heuristic_array / heuristic_array.max()
print(heuristic_array)

[0.30715574 0.         0.05542258 ... 0.04710137 0.04708409 0.04607807]


In [14]:
with open(
        "generality.score",
        "w") as f:
    for row in heuristic_array:
        f.write(str(row) + "\n")

In [15]:
np.savetxt("generality.heuristic.csv", heuristic_array)